## Colab commands

The following cells will download prerequisite for fewshot-face-translation-GAN.

In [ ]:
!git clone --recursive https://github.com/shaoanlu/fewshot-face-translation-GAN.git

Cloning into 'fewshot-face-translation-GAN'...
remote: Enumerating objects: 780, done.
remote: Counting objects: 100% (780/780), done.
remote: Compressing objects: 100% (760/760), done.
remote: Total 780 (delta 68), reused 682 (delta 14), pack-reused 0
Receiving objects: 100% (780/780), 12.27 MiB | 20.90 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
# There are import errors under keras == 2.2.5
!pip install keras==2.2.4

In [ ]:
%cd "fewshot-face-translation-GAN"

/content/fewshot-face-translation-GAN


In [ ]:
# Download pre-trined weights
!gdown https://drive.google.com/uc?id=1DUMmZGTGKMyEYSKy-w34IDHawVF24rIs
!gdown https://drive.google.com/uc?id=1xl8cg7xaRnMsyiODcXguJ83d5hwodckB

Downloading...
From: https://drive.google.com/uc?id=1DUMmZGTGKMyEYSKy-w34IDHawVF24rIs
To: /content/fewshot-face-translation-GAN/encoder.h5
6.26MB [00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xl8cg7xaRnMsyiODcXguJ83d5hwodckB
To: /content/fewshot-face-translation-GAN/decoder.h5
124MB [00:01, 93.5MB/s]


In [ ]:
!mkdir weights
!mv decoder.h5 weights/decoder.h5
!mv encoder.h5 weights/encoder.h5

In [ ]:
!ls

face_toolbox_keras  images  models.py  networks  README.md  utils  weights


## Load GAN model

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from models import FaceTranslationGANInferenceModel

Using TensorFlow backend.


In [ ]:
model = FaceTranslationGANInferenceModel()

Instructions for updating:
Colocations handled automatically by placer.
Found checkpoints in weights folder. Built model with pre-trained weights.


## Load face-toolbox

In [ ]:
from face_toolbox_keras.models.verifier.face_verifier import FaceVerifier
fv = FaceVerifier(classes=512)

from face_toolbox_keras.models.parser import face_parser
fp = face_parser.FaceParser()

from face_toolbox_keras.models.detector import face_detector
fd = face_detector.FaceAlignmentDetector()

from face_toolbox_keras.models.detector.iris_detector import IrisDetector
idet = IrisDetector()
#idet.set_detector(fd)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


# Translate faces

In [ ]:
import numpy as np
from utils import utils
from matplotlib import pyplot as plt

### Set path to input images

In [ ]:
from google.colab import files

In [ ]:
# Upload a source face image.
# There should be only one source face image.
fn_src = files.upload()

Saving test02.jpg to test02.jpg


In [ ]:
# Upload target face images.
# Number of target face images is not restricted as long as they belong to the same identity.
fns_tar = files.upload()

Saving test01.jpg to test01.jpg
Saving test03.jpg to test03.jpg
Saving test04.jpg to test04.jpg
Saving test05.jpg to test05.jpg


In [ ]:
fn_src = [k for k,v in fn_src.items()]
if len(fn_src) >= 1:
    fn_src = fn_src[0]

fns_tar = [k for k,v in fns_tar.items()]

print(fn_src)
print(fns_tar)

In [ ]:
# We can also manually assign filenmes
#fn_src = "test02.jpg"
#fns_tar = ["test01.jpg", "test03.jpg", "test04.jpg", "test05.jpg"]

### Inferece

It requires additional time to load models for the first infernce.

In [ ]:
src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(fn_src, fd, fp, idet)
tar, emb_tar = utils.get_tar_inputs(fns_tar, fd, fv)

Building FAN for landmarks detection...
Done.


In [ ]:
out = model.inference(src, mask, tar, emb_tar)

### Visualize results

Images are resized to having maximum side length of 768.

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(src)
plt.subplot(1,2,2)
plt.imshow(tar)

In [ ]:
result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
plt.imshow(result_face)

In [ ]:
result_img = utils.post_process_result(fn_src, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
plt.imshow(result_img)